In [1]:
DATA_NAME = 'coco-indoor' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_horizontal_wavelet_indoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,6559.04,0.13,0.04,0.03,0.00,0.00,-0.00,-0.00
3,0.13,1468.42,-0.02,0.01,-0.00,0.00,-0.00,0.00
4,0.04,-0.02,320.17,0.00,0.00,-0.00,0.00,-0.00
5,0.03,0.01,0.00,60.57,0.00,0.00,0.00,-0.00
6,0.00,-0.00,0.00,0.00,10.98,0.00,0.00,0.00
7,0.00,0.00,-0.00,0.00,0.00,1.92,-0.00,0.00
8,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.36,0.00
9,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.08


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00004,0.00003,0.00005,0.00001,0.00000,-0.00000,-0.00002
3,0.00004,1.00000,-0.00002,0.00004,-0.00001,0.00002,-0.00005,0.00009
4,0.00003,-0.00002,1.00000,0.00002,0.00001,-0.00006,0.00007,-0.00006
5,0.00005,0.00004,0.00002,1.00000,0.00002,0.00002,0.00002,-0.00004
6,0.00001,-0.00001,0.00001,0.00002,1.00000,0.00002,0.00003,0.00004
7,0.00000,0.00002,-0.00006,0.00002,0.00002,1.00000,-0.00004,0.00001
8,-0.00000,-0.00005,0.00007,0.00002,0.00003,-0.00004,1.00000,0.00003
9,-0.00002,0.00009,-0.00006,-0.00004,0.00004,0.00001,0.00003,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.20441465712846424

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[5.38362381e+06 2.64133017e+05 1.22048005e+04 4.19614777e+02
 1.29418733e+01 3.52319491e-01 9.38663446e-03 9.05061088e-35]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999411,-0.033573,-0.006919,-0.001302,-0.000237,-0.000041,-0.000008,-0.000002
1,0.033274,0.998687,-0.038334,-0.006854,-0.001242,-0.000216,-0.000041,-0.000008
2,0.008128,0.037775,0.998450,-0.039441,-0.006854,-0.001201,-0.000220,-0.000049
3,0.001833,0.008216,0.038768,0.998074,-0.047012,-0.007881,-0.001459,-0.000316
4,0.000415,0.001856,0.008518,0.046124,0.997026,-0.060129,-0.010713,-0.002303
5,0.000096,0.000427,0.001966,0.010367,0.058346,0.993470,-0.095491,-0.019550
6,0.000025,0.000114,0.000518,0.002746,0.015161,0.088398,0.972888,-0.213161
7,0.000011,0.000050,0.000234,0.001236,0.006811,0.039029,0.210366,0.976819


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0005885645042760235,
 0.0013134969052126522,
 0.001549745041348305,
 0.0019259940011777665,
 0.002973991719278146,
 0.006529607429266671,
 0.027112291537031208,
 0.02318128641135453]